# **Bayesian Personalized Ranking (BPR) Recommender on RetailRocket Data**

Authored by: Preet Khowaja

In this notebook we train a BPR model to recomend a ranked list of items to users in the dataset. We are using RetailRocket's e-commerce data to train this model. 

The BPR implementation used is from the cornac package in Python.:

In [26]:
## Install dependencies
# !pip install cornac
# !pip install papermill
# !pip install scrapbook
# !pip install recommenders


In [27]:
## Import required packages
import sys
import os
import cornac
import papermill as pm
import scrapbook as sb
import pandas as pd
import numpy as np

from recommenders.datasets.python_splitters import python_random_split
from recommenders.evaluation.python_evaluation import map_at_k, ndcg_at_k, precision_at_k, recall_at_k
from recommenders.models.cornac.cornac_utils import predict_ranking
from recommenders.utils.timer import Timer
from recommenders.utils.constants import SEED

## check what version of cornac is available
print("System version: {}".format(sys.version))
print("Cornac version: {}".format(cornac.__version__))

System version: 3.8.16 (default, Dec  7 2022, 01:12:13) 
[GCC 7.5.0]
Cornac version: 1.14.2


## Loading RetailRocket Data

1. Mount this notebook to Google Drive

2. Save the **zipped** events.csv file to somewhere in your drive from the RetailRocket data available [here](https://www.kaggle.com/datasets/retailrocket/ecommerce-dataset)

3. Change the command below to reflect the path where your zipped file is saved. It will unzip the file so it is only available during run-time

In [28]:
## Unzip events file of Retail Rocket and store in local directory
# !unzip drive/My\ Drive/aipi/final_project/RR_events.csv.zip

In [29]:
## Read the dataset
df = pd.read_csv('events.csv')
df.head()

,timestamp,visitorid,event,itemid,transactionid
0,1433221332117,257597,view,355908,NaN
1,1433224214164,992329,view,248676,NaN
2,1433221999827,111016,view,318965,NaN
3,1433221955914,483717,view,253185,NaN
4,1433221337106,951259,view,367447,NaN


In [30]:
## The event column shows us the feedback between user and item pairings in the dataset
df.event.value_counts()

view           2664312
addtocart        69332
transaction      22457
Name: event, dtype: int64

In [31]:
## Take a random sample from the full data because it's massive
df = df.sample(n=5000, random_state=45)

## Adding Ranking information to run BPR

BPR relies on some type of ranking for each user-item pair. Here we assume that if a user interacted with an item, the item is ranked as 1. Otherwise, it is ranked as 0. In our dataset we only have positive feedback available, so we generate the negative feedback.

In [32]:
## Assign a score of 1 to user-item interactions that are available
rr_data = df[['visitorid', 'itemid']].copy()
rr_data['Feedback'] = 1
rr_data = rr_data.drop_duplicates()

In [33]:
rr_data.head()

,visitorid,itemid,Feedback
2519133,961663,27127,1
2052038,1206986,435421,1
522082,437200,170460,1
2496162,731484,26210,1
2241508,1011436,401802,1


In [34]:
# Rename the columns for consistency 
rr_data.rename(columns = {'visitorid': 'userID', 'itemid': 'itemID', 'Feedback': 'rating'}, inplace = True)

In [35]:
## Create a list of unique users and unique items from our sample
users = rr_data['userID'].unique()
items = rr_data['itemID'].unique()

In [36]:
## Adding negative feedback for interactions not present
interaction_lst = []
for user in users:
    for item in items:
        interaction_lst.append([user, item, 0])

## Dataframe created for all negative feedback, i.e. where user has not interacted with the item
rr_data_all = pd.DataFrame(data=interaction_lst, columns=["userID", "itemID", "rating"])

In [37]:
## Check if the rating column has 0
rr_data_all.head()

,userID,itemID,rating
0,961663,27127,0
1,961663,435421,0
2,961663,170460,0
3,961663,26210,0
4,961663,401802,0


In [38]:
## Merge the datasets with positive and negative feedback
rr_feedback = pd.merge(rr_data_all, rr_data, on=['userID', 'itemID'], how='outer').fillna(0).drop('rating_x', axis = 1)

In [39]:
# Cleaning up the column names
rr_feedback.rename(columns = {'rating_y': 'rating'}, inplace = True)

In [40]:
## Check how many positive and negative feedback signals we have
## We should have more 0's because the users interact with fewer items 
rr_feedback['rating'].value_counts()

0.0    22170589
1.0        4993
Name: rating, dtype: int64

In [41]:
## Check how many observations
rr_feedback.shape

(22175582, 3)

In [42]:
## Split data into training and testing
train_rr, test_rr = python_random_split(rr_feedback, 0.8)

Initiate BPR Model and Train on Dataset

In [43]:
## Initiating a BPR model 
bpr = cornac.models.BPR(
    k=20,
    max_iter=30,
    learning_rate=0.01,
    lambda_reg=0.001,
    verbose=True,
    seed=43
)

The next cell takes about 50 seconds to run and a little bit of RAM

In [44]:
train_set_rr = cornac.data.Dataset.from_uir(train_rr.itertuples(index=False), seed=4747)
print('Number of users: {}'.format(train_set_rr.num_users))
print('Number of items: {}'.format(train_set_rr.num_items))


Number of users: 4862
Number of items: 4561


In [45]:
## Training the BPR model on our data
with Timer() as t:
    bpr.fit(train_set_rr)
print("Took {} seconds for training.".format(t))

  0%|          | 0/30 [00:00<?, ?it/s]

Optimization finished!
Took 247.5193 seconds for training.


In [46]:
with Timer() as t:
    all_predictions = predict_ranking(bpr, train_rr, usercol='userID', itemcol='itemID', remove_seen = False)
print("Took {} seconds for prediction.".format(t))

Took 12.6931 seconds for prediction.


In [47]:
## Each user-item pairing is given a prediction 
## This is basically an item's rated value by the user and 
## a ranked item's list for the user
all_predictions.head()

,userID,itemID,prediction
0,198803,294666,-0.181013
1,198803,366186,-0.033933
2,198803,290390,0.012200
3,198803,63405,-0.489395
4,198803,384556,-0.375238


In [48]:
## For top 5 recommendations here are the computed evaluation metrics:
k = 10
## Mean Average Precision
eval_map = map_at_k(test_rr, all_predictions, col_prediction='prediction', k=k)
## NDCG
eval_ndcg = ndcg_at_k(test_rr, all_predictions, col_prediction='prediction', k=k)
## Precision
eval_precision = precision_at_k(test_rr, all_predictions, col_prediction='prediction', k=k)
## Recall
eval_recall = recall_at_k(test_rr, all_predictions, col_prediction='prediction', k=k)

print("MAP:\t%f" % eval_map
     ,
      "NDCG:\t%f" % eval_ndcg,
      "Precision@K:\t%f" % eval_precision,
      "Recall@K:\t%f" % eval_recall, sep='\n'
)

MAP:	0.000318
NDCG:	0.084172
Precision@K:	0.086693
Recall@K:	0.000950


# **References**

1. Microsoft Recommenders, BPR Deep Dive
https://github.com/microsoft/recommenders/blob/main/examples/02_model_collaborative_filtering/cornac_bpr_deep_dive.ipynb

2. Microsoft Recommenders Preparing Data
https://github.com/microsoft/recommenders/blob/main/examples/01_prepare_data/data_transform.ipynb

3. Aghiles Salah, Quoc-Tuan Truong, Hady W. Lauw; *\"Cornac: A Comparative Framework for Multimodal
Recommender Systems* ; Journal of Machine Learning Research 2021 (2020) 1-5. 
https://dl.acm.org/doi/pdf/10.5555/3455716.3455811